# Asset Data

In [1]:
import ast
import numpy as np
import pandas as pd
from warnings import filterwarnings
from tqdm.notebook import tqdm
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
from pprint import pprint
from datetime import date

filterwarnings('ignore')

from src.contract_export import contract_query
from config import CONTRACT_ADDRESSES, NODE_LCD_URLS

init_notebook_mode(connected=True)
tqdm.pandas()

CONTRACT_CHAIN = 'osmosis-testnet'

## load data

In [2]:
assets_df = pd.read_csv('data_csv/all_assets.csv', index_col=0)
assets_df['channels'] = assets_df.channels.map(lambda x: ast.literal_eval(x) if type(x) == str else None)
assets_df['denom_units'] = assets_df.denom_units.map(
    lambda x: ast.literal_eval(x) if type(x) == str else None)
assets_df

,chain_id,denom,type_asset,supply,description,denom_units,display,name,symbol,uri,...,path,channels,chain_id_counterparty,supply_base,chain_id_base,channels_number,admin,ibc,traces,chain_name
0,osmosis-1,ibc/56A78E186FE5E87B7A6942ADCD1E319CF22C8C95E0...,ics20,1000000,The native token of Osmosis,"[{'denom': 'uosmo', 'exponent': 0, 'aliases': ...",osmo,NaN,NaN,NaN,...,transfer/channel-143/transfer/channel-0/transf...,"[channel-143, channel-0, channel-141, channel-...",kava_2222-10,624621394204877,osmosis-1,16,NaN,"{'source_channel': 'channel-143', 'dst_channel...","[{'type': 'ibc', 'counterparty': {'chain_id': ...",osmosis
1,kava_2222-10,ibc/B2719736BC8BE3AF15336EBEDD5FED70B5D9377CB5...,ics20,1000000,The native token of Osmosis,"[{'denom': 'uosmo', 'exponent': 0, 'aliases': ...",osmo,NaN,NaN,NaN,...,transfer/channel-0/transfer/channel-141/transf...,"[channel-0, channel-141, channel-42, channel-1...",cosmoshub-4,624621394204877,osmosis-1,15,NaN,"{'source_channel': 'channel-0', 'dst_channel':...","[{'type': 'ibc', 'counterparty': {'chain_id': ...",kava
2,cosmoshub-4,ibc/2C6714D652D104FFA9F45F034001A3A6AC3E062115...,ics20,1000000,The native token of Osmosis,"[{'denom': 'uosmo', 'exponent': 0, 'aliases': ...",osmo,NaN,NaN,NaN,...,transfer/channel-141/transfer/channel-42/trans...,"[channel-141, channel-42, channel-1, channel-1...",osmosis-1,624621394204877,osmosis-1,14,NaN,"{'source_channel': 'channel-141', 'dst_channel...","[{'type': 'ibc', 'counterparty': {'chain_id': ...",cosmoshub
3,osmosis-1,ibc/773753BB5351F04757EFD94E7CD4955B9EB55069C8...,ics20,1000000,The native token of Osmosis,"[{'denom': 'uosmo', 'exponent': 0, 'aliases': ...",osmo,NaN,NaN,NaN,...,transfer/channel-42/transfer/channel-1/transfe...,"[channel-42, channel-1, channel-141, channel-1...",juno-1,624621394204877,osmosis-1,13,NaN,"{'source_channel': 'channel-42', 'dst_channel'...","[{'type': 'ibc', 'counterparty': {'chain_id': ...",osmosis
4,acre_9052-1,ibc/FFC19355CBEABDBA8C03703F9E552FFF22751B3288...,ics20,50000000000000000,Cosmos coin token representation of 0xAE6D3334...,[{'denom': 'erc20/0xAE6D3334989a22A65228732446...,CientoToken,erc20/0xAE6D3334989a22A65228732446731438672418F2,CNTO,NaN,...,transfer/channel-0/transfer/channel-491/transf...,"[channel-0, channel-491, channel-0, channel-20...",osmosis-1,399646151476330794095721,acre_9052-1,12,NaN,"{'source_channel': 'channel-0', 'dst_channel':...","[{'type': 'ibc', 'counterparty': {'chain_id': ...",acrechain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13669,phoenix-1,cw20:terra1x62mjnme4y0rdnag3r8rfgjuutsqlkkyuh4...,cw20,167455604142269,NaN,"[{'denom': 'xastro', 'exponent': 6, 'aliases':...",NaN,Staked Astroport,xASTRO,NaN,...,NaN,None,NaN,167455604142269,phoenix-1,0,NaN,NaN,NaN,terra2
13670,phoenix-1,cw20:terra17gck626vgax9jpe6utm7dhx4vdzawfkt0jh...,cw20,93000000000000,NaN,"[{'denom': 'dust', 'exponent': 6, 'aliases': [...",NaN,Depegged UST,dUST,NaN,...,NaN,None,NaN,93000000000000,phoenix-1,0,NaN,NaN,NaN,terra2
13671,phoenix-1,cw20:terra17aj4ty4sz4yhgm08na8drc0v03v2jwr3wax...,cw20,1540209048927,NaN,"[{'denom': 'bluna', 'exponent': 6, 'aliases': ...",NaN,boneLuna,bLUNA,NaN,...,NaN,None,NaN,1540209048927,phoenix-1,0,NaN,NaN,NaN,terra2
13672,phoenix-1,cw20:terra1xzkel96e5e8vfmqw7valzdzzv9hqasfyslc...,cw20,7088687608574,NaN,"[{'denom': 'local', 'exponent': 6, 'aliases': ...",NaN,Local Terra Token (Wormhole),LOCAL,NaN,...,NaN,None,NaN,7088687608574,phoenix-1,0,NaN,NaN,NaN,terra2


## charts

In [3]:
def aggregate_pie_plot(assets_df: pd.DataFrame, agg_column: str, n: int = 8, object_name: str = 'items') -> pd.Series:
    assets_by_item_row = assets_df.groupby(agg_column)['denom'].agg(np.count_nonzero).sort_values(ascending=False)
    assets_by_item = assets_by_item_row.head(n)
    if len(assets_by_item_row) > n:
        assets_by_item[f'remaining {len(assets_by_item_row) - n:>,} {object_name}<br>'] = sum(assets_by_item_row[n:])
    return assets_by_item

In [4]:
ASSET_TYPE_DICT = \
    {'ics20': 'IBC (ICS-20)',
     'sdk.coin': 'Native',
     'pool': 'LP Tokens',
     'factory': 'Token Factory',
     'cw20': 'CW-20',
     'erc20': 'ERC-20'}

assets_by_chain = aggregate_pie_plot(assets_df=assets_df, agg_column='chain_name', object_name='chains\n', n=17)

assets_by_type_asset = aggregate_pie_plot(assets_df=assets_df, agg_column='type_asset')
assets_by_type_asset.index = assets_by_type_asset.index.map(lambda x: ASSET_TYPE_DICT[x])

assets_by_denom_base = aggregate_pie_plot(assets_df=assets_df, agg_column='denom_base', object_name='base denoms\n',
                                          n=6)

total_assets = len(assets_df)

In [5]:
fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type': 'pie', 'colspan': 2}, None],
           [{'type': 'pie'}, {'type': 'pie'}]],
    subplot_titles=('Cosmos-SDK Assets by Chains', 'by Type', 'by Base Denoms'),
    horizontal_spacing=0.4,
    vertical_spacing=0.14
)

fig.add_trace(
    go.Pie(
        labels=list(assets_by_chain.to_dict().keys()),
        values=list(assets_by_chain.to_dict().values()),
        text=list(assets_by_chain.map('{:>,}'.format).to_dict().values()),
        sort=False,
        name='assets by chains',
        rotation=-35,
        marker_colors=px.colors.qualitative.Plotly
    ),
    row=1,
    col=1)

fig.add_trace(
    go.Pie(
        labels=list(assets_by_type_asset.to_dict().keys()),
        values=list(assets_by_type_asset.to_dict().values()),
        text=list(assets_by_type_asset.map('{:>,}'.format).to_dict().values()),
        sort=False,
        name='assets by type',
        rotation=90,
        marker_colors=px.colors.qualitative.G10
    ),
    row=2,
    col=1)
fig.add_trace(
    go.Pie(
        labels=list(assets_by_denom_base.to_dict().keys()),
        values=list(assets_by_denom_base.to_dict().values()),
        text=list(assets_by_denom_base.map('{:>,}'.format).to_dict().values()),
        sort=False,
        name='assets by base denoms',
        rotation=75,
        marker_colors=px.colors.qualitative.Set1_r
    ),
    row=2,
    col=2)
fig.update_traces(
    textposition='outside',
    hoverinfo='label+value+percent',
    textinfo='label+value+percent',
    texttemplate=' %{label}  %{value:>,} (%{percent:.1%}) ',
    hovertemplate='%{label}<br>%{value:>,} (%{percent:.1%})'
)
fig.update_layout(
    height=850,
    width=1000,
    showlegend=False,
    font_family="Play",
    title_text='',
    title_x=0.5,
    template='plotly_white',
    autosize=False,
    margin={'l': 0, 'r': 0, 't': 80, 'b': 0}
)

fig['layout']['annotations'][0]['y'] = 1.04
fig['layout']['annotations'][1]['y'] = 0.37
fig['layout']['annotations'][2]['y'] = 0.37

fig.add_annotation(
    x=-0.3,
    y=-0.05,
    text=f'github.com/Snedashkovsky/on-chain-registry',
    showarrow=False)
fig.add_annotation(
    x=1.25,
    y=-0.05,
    text=f'updated {date.today()}',
    showarrow=False)

fig.write_image('charts/charts.png', scale=1.5)
fig.write_html('charts/index.html', include_plotlyjs='cdn')
iplot(fig)

## contract query
### get any assets in alphabetical order for a given chain

In [6]:
# Get any `4` assets in the `Osmosis` chain,  
pprint(
    contract_query(
        query={'get_assets_by_chain':
                   {'chain_name': 'osmosis',
                    'limit': 4,
                    'start_after_base': 'ibc/FF6F4774ABC2478832A6D6681DB9A7A8DDC4485212A8AED84642EFCD63C748A9'}
               },
        contract_address=CONTRACT_ADDRESSES[CONTRACT_CHAIN],
        node_lcd_url=NODE_LCD_URLS[CONTRACT_CHAIN],
        display_query=True
    )
)

31-10-2023 02:07:22 - INFO - https://osmosis-testnet-api.polkachu.com/cosmwasm/wasm/v1/contract/osmo1nwesd2xe6cnvtpqd29xg7qeznlm65x02lfjfg20wlvkdze20hcxsftxtzz/smart/eyJnZXRfYXNzZXRzX2J5X2NoYWluIjogeyJjaGFpbl9uYW1lIjogIm9zbW9zaXMiLCAibGltaXQiOiA0LCAic3RhcnRfYWZ0ZXJfYmFzZSI6ICJpYmMvRkY2RjQ3NzRBQkMyNDc4ODMyQTZENjY4MURCOUE3QThEREM0NDg1MjEyQThBRUQ4NDY0MkVGQ0Q2M0M3NDhBOSJ9fQ==
{'data': {'assets': [{'address': None,
                      'admin': None,
                      'base': 'ibc/FFA652599C77E853F017193E36B5AB2D4D9AFC4B54721A74904F80C9236BF3B7',
                      'chain_id': 'osmosis-1',
                      'chain_name': 'osmosis',
                      'coingecko_id': None,
                      'denom_units': None,
                      'description': None,
                      'display': None,
                      'ibc': {'base_supply': None,
                              'dst_channel': 'channel-3',
                              'source_channel': 'channel-286',
            

### get any assets from any chains

In [7]:
# get any `5` assets from any chains
pprint(
    contract_query(
        query={'get_all_assets':
                   {'limit': 5}
               },
        contract_address=CONTRACT_ADDRESSES[CONTRACT_CHAIN],
        node_lcd_url=NODE_LCD_URLS[CONTRACT_CHAIN],
        display_query=True
    )
)

31-10-2023 02:07:22 - INFO - https://osmosis-testnet-api.polkachu.com/cosmwasm/wasm/v1/contract/osmo1nwesd2xe6cnvtpqd29xg7qeznlm65x02lfjfg20wlvkdze20hcxsftxtzz/smart/eyJnZXRfYWxsX2Fzc2V0cyI6IHsibGltaXQiOiA1fX0=
{'data': {'entries': [{'address': None,
                       'admin': None,
                       'base': 'ibc/4A3AAD07BC4EBEBC10FC2560EAA3B7A1D3B541B5264ED8E5E13E6B74AC76127B',
                       'chain_id': 'eightball-1',
                       'chain_name': '8ball',
                       'coingecko_id': None,
                       'denom_units': [{'aliases': [],
                                        'denom': 'uosmo',
                                        'exponent': 0},
                                       {'aliases': [],
                                        'denom': 'osmo',
                                        'exponent': 6}],
                       'description': 'The native token of Osmosis',
                       'display': 'osmo',
                  

### get asset for a given denom and a given chain

In [8]:
# get asset for `ibc/E86611817EAF331173D204962298A16C236E918F7C7E985497A63232B926F162` (ATOM from Osmosis in Crescent) denom in `Crescent` chain
pprint(
    contract_query(
        query={'get_asset':
                   {'chain_name': 'crescent',
                    'base': 'ibc/E86611817EAF331173D204962298A16C236E918F7C7E985497A63232B926F162'}
               },
        contract_address=CONTRACT_ADDRESSES[CONTRACT_CHAIN],
        node_lcd_url=NODE_LCD_URLS[CONTRACT_CHAIN],
        display_query=True
    )
)

31-10-2023 02:07:25 - INFO - https://osmosis-testnet-api.polkachu.com/cosmwasm/wasm/v1/contract/osmo1nwesd2xe6cnvtpqd29xg7qeznlm65x02lfjfg20wlvkdze20hcxsftxtzz/smart/eyJnZXRfYXNzZXQiOiB7ImNoYWluX25hbWUiOiAiY3Jlc2NlbnQiLCAiYmFzZSI6ICJpYmMvRTg2NjExODE3RUFGMzMxMTczRDIwNDk2MjI5OEExNkMyMzZFOTE4RjdDN0U5ODU0OTdBNjMyMzJCOTI2RjE2MiJ9fQ==
{'data': {'asset': {'address': None,
                    'admin': None,
                    'base': 'ibc/E86611817EAF331173D204962298A16C236E918F7C7E985497A63232B926F162',
                    'chain_id': 'crescent-1',
                    'chain_name': 'crescent',
                    'coingecko_id': None,
                    'denom_units': [{'aliases': ['microatom'],
                                     'denom': 'uatom',
                                     'exponent': 0},
                                    {'aliases': ['milliatom'],
                                     'denom': 'matom',
                                     'exponent': 3},
                     